In [3]:
import requests
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
# from config import weather_api_key

In [4]:
api_key = '381f4cd9a71ed2a3e2bb3515a1a03590'

In [5]:
url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?"
units = "imperial"

In [7]:
uber_csv = pd.read_csv("Resources/uber.csv")

uber_df = pd.DataFrame(uber_csv)

uber_df.head()



,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [8]:
uber_df.rename(columns = {'Unnamed: 0':'ID', 'key':'N/A', 'fare_amount':'Charge', "pickup_datetime":"Pickup Time",  "pickup_longitude": "Pickup Lon", "pickup_latitude":"Pickup Lat",  "dropoff_longitude": "Dropoff Lon", "dropoff_latitude": "Dropoff Lat", "passenger_count": "Passenger Count"}, inplace = True)

uber_df.head()



,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [9]:
uber_df3 = uber_df.sample(n=5)
uber_df3

,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count
90155,51876687,2012-04-12 08:39:00.000000271,10.5,2012-04-12 08:39:00 UTC,-73.948758,40.789082,-73.953877,40.767110,1
171684,51640096,2012-01-29 20:16:00.00000045,5.3,2012-01-29 20:16:00 UTC,-73.968090,40.759057,-73.978415,40.763680,5
28392,42189179,2014-11-17 22:35:27.0000001,7.0,2014-11-17 22:35:27 UTC,-73.992460,40.756111,-73.988602,40.743731,1
45213,9734174,2013-06-14 12:20:16.0000004,10.5,2013-06-14 12:20:16 UTC,-73.962631,40.770288,-73.976783,40.776491,1
111035,10550862,2009-08-16 22:39:00.00000020,8.9,2009-08-16 22:39:00 UTC,-73.971152,40.754755,-74.002585,40.744582,2


In [10]:
uber_df3['Date Formatted'] = (pd.to_datetime(uber_df3['Pickup Time'], yearfirst=True, utc=True)) 
x = (uber_df3["Date Formatted"] - pd.Timestamp("1970-01-01", tz="UTC")) // pd.Timedelta('1s') 
uber_df3


,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count,Date Formatted
90155,51876687,2012-04-12 08:39:00.000000271,10.5,2012-04-12 08:39:00 UTC,-73.948758,40.789082,-73.953877,40.767110,1,2012-04-12 08:39:00+00:00
171684,51640096,2012-01-29 20:16:00.00000045,5.3,2012-01-29 20:16:00 UTC,-73.968090,40.759057,-73.978415,40.763680,5,2012-01-29 20:16:00+00:00
28392,42189179,2014-11-17 22:35:27.0000001,7.0,2014-11-17 22:35:27 UTC,-73.992460,40.756111,-73.988602,40.743731,1,2014-11-17 22:35:27+00:00
45213,9734174,2013-06-14 12:20:16.0000004,10.5,2013-06-14 12:20:16 UTC,-73.962631,40.770288,-73.976783,40.776491,1,2013-06-14 12:20:16+00:00
111035,10550862,2009-08-16 22:39:00.00000020,8.9,2009-08-16 22:39:00 UTC,-73.971152,40.754755,-74.002585,40.744582,2,2009-08-16 22:39:00+00:00


In [11]:
uber_df3['Date Formatted Final'] = (uber_df3["Date Formatted"] - pd.Timestamp("1970-01-01", tz="UTC")) // pd.Timedelta('1s')
uber_df3



,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count,Date Formatted,Date Formatted Final
90155,51876687,2012-04-12 08:39:00.000000271,10.5,2012-04-12 08:39:00 UTC,-73.948758,40.789082,-73.953877,40.767110,1,2012-04-12 08:39:00+00:00,1334219940
171684,51640096,2012-01-29 20:16:00.00000045,5.3,2012-01-29 20:16:00 UTC,-73.968090,40.759057,-73.978415,40.763680,5,2012-01-29 20:16:00+00:00,1327868160
28392,42189179,2014-11-17 22:35:27.0000001,7.0,2014-11-17 22:35:27 UTC,-73.992460,40.756111,-73.988602,40.743731,1,2014-11-17 22:35:27+00:00,1416263727
45213,9734174,2013-06-14 12:20:16.0000004,10.5,2013-06-14 12:20:16 UTC,-73.962631,40.770288,-73.976783,40.776491,1,2013-06-14 12:20:16+00:00,1371212416
111035,10550862,2009-08-16 22:39:00.00000020,8.9,2009-08-16 22:39:00 UTC,-73.971152,40.754755,-74.002585,40.744582,2,2009-08-16 22:39:00+00:00,1250462340


In [12]:
clean_df = uber_df3[["ID", "Charge", "Pickup Lon", "Pickup Lat", "Dropoff Lon", "Dropoff Lat", "Passenger Count", "Date Formatted Final" ]]
clean_df


,ID,Charge,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count,Date Formatted Final
90155,51876687,10.5,-73.948758,40.789082,-73.953877,40.767110,1,1334219940
171684,51640096,5.3,-73.968090,40.759057,-73.978415,40.763680,5,1327868160
28392,42189179,7.0,-73.992460,40.756111,-73.988602,40.743731,1,1416263727
45213,9734174,10.5,-73.962631,40.770288,-73.976783,40.776491,1,1371212416
111035,10550862,8.9,-73.971152,40.754755,-74.002585,40.744582,2,1250462340


In [26]:
for index, row in clean_df.iterrows():

# for row in uber:
    lat = row['Pickup Lat']
    lon = row['Pickup Lon']
    date = row['Date Formatted Final'].astype(int)
    query_url = f"{url}lat={lat}&lon={lon}&dt={date}&appid={api_key}"
    print(query_url)
    print(lat)
    print(date)

https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.789082&lon=-73.948758&dt=1334219940&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.789082
1334219940
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.759057&lon=-73.96809&dt=1327868160&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.759057
1327868160
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.756111&lon=-73.99246&dt=1416263727&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.756111
1416263727
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.770288&lon=-73.962631&dt=1371212416&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.770288
1371212416
https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=40.754755&lon=-73.971152&dt=1250462340&appid=381f4cd9a71ed2a3e2bb3515a1a03590
40.754755
1250462340


In [27]:
response = requests.get(query_url)
response_json = response.json()

print(response_json)

{'lat': 40.7548, 'lon': -73.9712, 'timezone': 'America/New_York', 'timezone_offset': -14400, 'data': [{'dt': 1250462340, 'sunrise': 1250417252, 'sunset': 1250466775, 'temp': 301.59, 'feels_like': 303.67, 'pressure': 1020, 'humidity': 63, 'dew_point': 293.87, 'clouds': 0, 'visibility': 10000, 'wind_speed': 5.1, 'wind_deg': 160, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]}]}
